In [1]:
'\\u{:04d}'.format(0x2580)

'\\u9600'

In [2]:
chr(0x2580)

'▀'

In [3]:
' ' + ''.join([chr(n) for n in range(0x2581, 0x2588)])

' ▁▂▃▄▅▆▇'

In [4]:
print(" \n" + '\n'.join([chr(n) for n in range(0x258F, 0x2587, -1)]))

 
▏
▎
▍
▌
▋
▊
▉
█


In [5]:
import numpy as np

In [6]:
X = np.arange(-4., 4.25, 0.25)
X

array([-4.  , -3.75, -3.5 , -3.25, -3.  , -2.75, -2.5 , -2.25, -2.  ,
       -1.75, -1.5 , -1.25, -1.  , -0.75, -0.5 , -0.25,  0.  ,  0.25,
        0.5 ,  0.75,  1.  ,  1.25,  1.5 ,  1.75,  2.  ,  2.25,  2.5 ,
        2.75,  3.  ,  3.25,  3.5 ,  3.75,  4.  ])

In [7]:
Y = X * X
Y

array([16.    , 14.0625, 12.25  , 10.5625,  9.    ,  7.5625,  6.25  ,
        5.0625,  4.    ,  3.0625,  2.25  ,  1.5625,  1.    ,  0.5625,
        0.25  ,  0.0625,  0.    ,  0.0625,  0.25  ,  0.5625,  1.    ,
        1.5625,  2.25  ,  3.0625,  4.    ,  5.0625,  6.25  ,  7.5625,
        9.    , 10.5625, 12.25  , 14.0625, 16.    ])

In [8]:
np.round(16.3).astype(int)

np.int64(16)

In [78]:
def blockplot(
    Y : np.array, X : np.array, limits : bool = True, perzero : bool = False,
    x_limits : tuple = None, Y2 : np.array = None
) -> str:
    if Y.ndim != 1 or X.ndim != 1 or (Y2 is not None and Y2.ndim != 1):
        return "Inputs are not one-dimensional"
    elif Y.size != X.size or (Y2 is not None and Y2.size != X.size):
        return "Inputs are not the same size"

    if Y2 is not None:
        double_mode = True
    else:
        double_mode = False

    color1 = 21
    color2 = 93

    color1_prefix = "\x1B[38;5;{:d}m".format(color1)
    color2_prefix = "\x1B[38;5;{:d}m".format(color2)
    colors = (color1_prefix, color2_prefix)
    
    color1_to_2 = "\x1B[38;5;{:d};48;5;{:d}m".format(color1, color2)
    color2_to_1 = "\x1B[38;5;{:d};48;5;{:d}m".format(color2, color1)
    color_changes = (color1_to_2, color2_to_1) # index with smaller
    
    color_clear = "\x1B[0m"
    color_reverse = "\x1B[7m"

    if double_mode:
        ymin = min(np.min(Y), np.min(Y2))
        ymax = max(np.max(Y), np.max(Y2))
    else:
        ymin = np.min(Y)
        ymax = np.max(Y)

    if perzero:
        yrange = np.max(np.array([ymax - 0., ymax - ymin, 0 - ymin]))
    else:
        yrange = ymax - ymin
        Y -= ymin

    assert np.round(yrange, 3) != 0.

    blocks = [[' ' for x in range(X.size)] for y in range(round(yrange))]
    chars = ' ' + ''.join([chr(c) for c in range(0x2581, 0x2588)])
    #print(chars)heroes

    if double_mode:
        T = np.vstack((np.copy(Y), np.copy(Y2)))
    else:
        T = np.copy(Y)

    if perzero:
        print("top")
        for y in range(max(round(ymax), 0) - 1, -1, -1):
            print(y)
            for x in range(X.size):
                if double_mode:
                    i = smaller[x]
                    j = bigger[x]

                    if T[i, x] >= 1.:
                        blocks[y][x] = colors[i] + '\u2588' + color_clear
                        T[:, x] -= 1.
                    elif T[i, x] >= 0.:
                        change = color_changes[i]
                        blocks[y][x] = change + chars[int(8 * T[i, x])] + color_clear
                        T[i, x] = float("NaN")
                        if T[j, x] < 1.:
                            T[j, x] = float("NaN")
                    elif T[j, x] >= 1.:
                        blocks[y][x] = colors[j] + '\u2588' + color_clear
                        T[:, x] -= 1.
                    elif T[j, x] >= 0:
                        blocks[y][x] = colors[j] + chars[int(8 * T[j, x])] + color_clear
                        T[j, x] = float("NaN")
                        
                else:
                    if T[x] >= 1.:
                        blocks[y][x] = '\u2588'
                        T[x] -= 1.
                    elif T[x] >= 0.:
                        blocks[y][x] = chars[int(8 * T[x])]
                        T[x] = float("NaN")
            #if double_mode:
            #    print(T)
        print("bottom")
        for y in range(max(round(ymax), 0), len(blocks)):
            print(y)
            for x in range(X.size):
                if double_mode:
                    pass
                else:
                    if T[x] <= -1.:
                        blocks[y][x] = '\u2588'
                        T[x] += 1.
                    elif T[x] <= 0.:
                        blocks[y][x] = color_reverse + chars[7 - int(8 * T[x] * -1)] + color_clear
                        T[x] = float("NaN")
    else:
        for y in range(len(blocks) - 1, -1, -1):
            #print(y)
            for x in range(X.size):
                if double_mode:
                    pass
                else:
                    if T[x] >= 1.:
                        blocks[y][x] = '\u2588'
                        T[x] -= 1.
                    elif T[x] >= 0.:
                        #print(T[x])
                        blocks[y][x] = chars[int(8 * T[x])]
                        T[x] = float("NaN")
    
    if limits:
        for i in range(round(yrange)):
            blocks[i].insert(0, "        ")

        if perzero:
            blocks[0][0] = "{: ^7.2F} ".format(max(ymax, 0.))
            blocks[-1][0] = "{: ^7.2F} ".format(min(ymin, 0.))
        else:
            blocks[0][0] = "{: ^7.2F} ".format(ymax)
            blocks[-1][0] = "{: ^7.2F} ".format(ymin)
        
        blocks.append(
            ["        ", "{: <7.2F} ".format(np.min(X)), ' ' * max(X.size - 16, 0), " {: >7.2F}".format(np.max(X))]
        )
        if x_limits is not None and len(x_limits) == 2:
            blocks[-1][1] = "{: <8s}".format(x_limits[0][:8])
            blocks[-1][3] = "{: >8s}".format(x_limits[1][:8])
    
    return '\n'.join([''.join([char for char in line]) for line in blocks])

In [79]:
Y

array([16.    , 14.0625, 12.25  , 10.5625,  9.    ,  7.5625,  6.25  ,
        5.0625,  4.    ,  3.0625,  2.25  ,  1.5625,  1.    ,  0.5625,
        0.25  ,  0.0625,  0.    ,  0.0625,  0.25  ,  0.5625,  1.    ,
        1.5625,  2.25  ,  3.0625,  4.    ,  5.0625,  6.25  ,  7.5625,
        9.    , 10.5625, 12.25  , 14.0625, 16.    ])

In [80]:
X

array([-4.  , -3.75, -3.5 , -3.25, -3.  , -2.75, -2.5 , -2.25, -2.  ,
       -1.75, -1.5 , -1.25, -1.  , -0.75, -0.5 , -0.25,  0.  ,  0.25,
        0.5 ,  0.75,  1.  ,  1.25,  1.5 ,  1.75,  2.  ,  2.25,  2.5 ,
        2.75,  3.  ,  3.25,  3.5 ,  3.75,  4.  ])

In [81]:
print(blockplot(Y - 2., X))

 14.00  █                               █
        █                               █
        ██                             ██
        ██▂                           ▂██
        ███                           ███
        ███▄                         ▄███
        ████                         ████
        █████                       █████
        █████▄                     ▄█████
        ██████▂                   ▂██████
        ███████                   ███████
        ████████                 ████████
        █████████               █████████
        ██████████▂           ▂██████████
        ███████████▄         ▄███████████
 -2.00  █████████████▄▂   ▂▄█████████████
        -4.00                        4.00


In [82]:
print(blockplot(5. * np.sin(X) + 10., X, x_limits = ("foo", "bar")))

 14.99                      ▁▅▇▇▄        
        ▆                  ▃█████▇       
        █▆                ▃███████▇      
        ██▅              ▁█████████▇     
        ███▄             ███████████▅    
        ████▂           ▇████████████▃   
        █████          ▆██████████████▁  
        ██████        ▄████████████████▁ 
        ██████▇      ▄██████████████████▁
 5.01   ████████▃  ▁▆████████████████████
        foo                           bar


In [83]:
print(blockplot(5. * np.sin(X) + 10., X, x_limits = ("snafu", "gambolputty"), perzero = True))

top
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
bottom
 14.99                      ▁▅▇▇▄        
        ▆                  ▃█████▇       
        █▆                ▃███████▇      
        ██▆              ▁█████████▇     
        ███▄             ███████████▅    
        ████▂           █████████████▃   
        █████          ▆██████████████▁  
        ██████        ▄████████████████▁ 
        ███████      ▄██████████████████▁
        ████████▃  ▂▆████████████████████
        █████████████████████████████████
        █████████████████████████████████
        █████████████████████████████████
        █████████████████████████████████
 0.00   █████████████████████████████████
        snafu                    gambolpu


In [84]:
print(blockplot(5. * np.sin(X) - 10., X, perzero = True))

top
bottom
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
 0.00   █████████████████████████████████
        █████████████████████████████████
        █████████████████████████████████
        █████████████████████████████████
        █████████████████████████████████
        ████████████████████▁▅▇▇▄████████
        ▆██████████████████▃     ▇███████
         ▆████████████████▃       ▇██████
          ▆██████████████▁         ▇█████
           ▄█████████████           ▅████
            ▂███████████▇            ▃███
              █████████▆              ▁██
               ███████▄                ▁█
                █████▄                  ▁
-14.99          ▃  ▂▆                    
        -4.00                        4.00


In [85]:
print(blockplot(5. * np.sin(X) - 10., X, perzero = False))

 -5.01                      ▁▅▇▇▄        
        ▆                  ▃█████▇       
        █▆                ▃███████▇      
        ██▅              ▁█████████▇     
        ███▄             ███████████▅    
        ████▂           ▇████████████▃   
        █████          ▆██████████████▁  
        ██████        ▄████████████████▁ 
        ██████▇      ▄██████████████████▁
-14.99  ████████▃  ▁▆████████████████████
        -4.00                        4.00


In [86]:
4. * np.sin(X) + 8.

array([11.02720998, 10.28624527,  9.40313291,  8.43278054,  7.43551997,
        6.47335603,  5.60611142,  4.88770721,  4.36281029,  4.06405621,
        4.01002005,  4.20406152,  4.63411606,  5.27344496,  6.08229785,
        7.01038416,  8.        ,  8.98961584,  9.91770215, 10.72655504,
       11.36588394, 11.79593848, 11.98997995, 11.93594379, 11.63718971,
       11.11229279, 10.39388858,  9.52664397,  8.56448003,  7.56721946,
        6.59686709,  5.71375473,  4.97279002])

In [87]:
print(blockplot(5 * np.sin(X), X))

 4.99                       ▁▅▇▇▄        
        ▆                  ▃█████▇       
        █▆                ▃███████▇      
        ██▅              ▁█████████▇     
        ███▄             ███████████▅    
        ████▂           ▇████████████▃   
        █████          ▆██████████████▁  
        ██████        ▄████████████████▁ 
        ██████▇      ▄██████████████████▁
 -4.99  ████████▃  ▁▆████████████████████
        -4.00                        4.00


In [88]:
print(blockplot(5 * np.sin(X), X, perzero = True))

top
4
3
2
1
0
bottom
5
6
7
8
9
 4.99                       ▁▅▇▇▄        
        ▆                  ▃█████▇       
        █▆                ▃███████▇      
        ██▆              ▁█████████▇     
        ███▄             ███████████▅    
            ▂███████████             ▃███
              █████████▆              ▁██
               ███████▄                ▁█
                █████▄                  ▁
 -4.99          ▃  ▂▆                    
        -4.00                        4.00


In [89]:
print(blockplot(5 * np.sin(X) - 7., X, perzero = True))

top
bottom
0
1
2
3
4
5
6
7
8
9
10
11
 0.00   █████████████████████████████████
        █████████████████████████████████
        ████████████████████▁▅▇▇▄████████
        ▆██████████████████▃     ▇███████
         ▆████████████████▃       ▇██████
          ▆██████████████▁         ▇█████
           ▄█████████████           ▅████
            ▂███████████▇            ▃███
              █████████▆              ▁██
               ███████▄                ▁█
                █████▄                  ▁
-11.99          ▃  ▂▆                    
        -4.00                        4.00


In [90]:
print(blockplot(5 * np.sin(X) + 7., X, perzero = True))

top
11
10
9
8
7
6
5
4
3
2
1
0
bottom
 11.99                      ▁▅▇▇▄        
        ▆                  ▃█████▇       
        █▆                ▃███████▇      
        ██▆              ▁█████████▇     
        ███▄             ███████████▅    
        ████▂           █████████████▃   
        █████          ▆██████████████▁  
        ██████        ▄████████████████▁ 
        ███████      ▄██████████████████▁
        ████████▃  ▂▆████████████████████
        █████████████████████████████████
 0.00   █████████████████████████████████
        -4.00                        4.00


In [91]:
chars = ' ' + ''.join([chr(n) for n in range(0x2581, 0x2588)])
chars

' ▁▂▃▄▅▆▇'

In [92]:
print("\x1B[7m" + chars + "\x1B[0m")

 ▁▂▃▄▅▆▇


In [100]:
print(blockplot(X * X, X, perzero = True, Y2 = 2. * np.abs(X)))

top
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
bottom
 16.00  █                               █
        ██                             ██
        ██▂                           ▂██
        ███                           ███
        ███▄                         ▄███
        ████                         ████
        █████                       █████
         ████▄                     ▄████ 
        █▄ ███▂                   ▂███ ▄█
        ███▄ ██                   ██ ▄███
        █████▄ █                 █ ▄█████
        ███████▄                 ▄███████
        █████████ █▄         ▄█ █████████
        ██████████▂██▄     ▄██▂██████████
        ███████████▄ ██   ██ ▄███████████
 0.00   █████████████▄▂   ▂▄█████████████
        -4.00                        4.00
